延續剛剛的 ResNet 模型來實作遷移學習，並進行微調。在這個實作中，我們將使用 TensorFlow 和 Keras 提供的 **ResNet50** 預訓練模型，並應用在自定義的資料集上。這裡的資料集可以是任何圖像分類的數據集，例如 CIFAR-10 或你自己的數據集。

---

### **遷移學習與微調 ResNet 預訓練模型**

我們將使用 ResNet50，它是預先在 **ImageNet** 數據集上訓練的，並通過微調（Fine-tuning）使它適應新的自定義資料集。這裡我們以 CIFAR-10 作為例子，但你可以替換為其他自定義數據集。

---

### **步驟**：

1. **導入 ResNet50 預訓練模型**
2. **凍結部分卷積層**
3. **替換輸出層為自定義類別數**
4. **進行微調**

---

### **實作重點**：

1. **使用預訓練的 ResNet50**：
   我們使用了在 ImageNet 上預訓練的 ResNet50 作為特徵提取器，通過 `include_top=False` 去掉最後的分類層，以適應 CIFAR-10 的圖像尺寸和類別數。

2. **凍結卷積層**：
   初始訓練時，我們將 ResNet50 的卷積層凍結，這意味著這些層的權重保持不變，僅訓練新的分類層。這樣可以保留預訓練模型中已經學到的圖像特徵，並快速訓練。

3. **自定義分類層**：
   在 ResNet50 的基礎上，我們添加了一個全局平均池化層和兩個全連接層，其中包含 Dropout 技術來防止過擬合。

4. **微調模型**：
   訓練完新的分類層後，我們進行微調，解凍 ResNet50 的最後 10 層，並使用較小的學習率來更新這些層的權重。這有助於模型更加適應 CIFAR-10 的特徵。

---

### **實作：ResNet50 遷移學習應用於 CIFAR-10**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# 下載並預處理 CIFAR-10 資料集
# CIFAR-10 是一個常用的圖像分類資料集。
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 正規化數據
# 將圖像數據正規化到 [0, 1] 範圍，以提高模型的訓練效果。
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 將標籤轉換為 one-hot 編碼，以便在訓練過程中使用 categorical_crossentropy 損失函數。
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 載入 ResNet50 模型，指定使用 ImageNet 預訓練權重，去掉頂層（即不包括最後的分類層）
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# 凍結預訓練模型的卷積層
# 凍結預訓練模型的所有卷積層，這樣在訓練過程中這些層的權重不會被更新。
for layer in base_model.layers:
    layer.trainable = False

# 建立新的分類層
# 在預訓練模型的基礎上添加新的分類層，包括全局平均池化層、一個全連接層、Dropout 層和最終的分類層。
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # 使用全局平均池化層代替展開層
    layers.Dense(128, activation='relu'),  # 添加一個全連接層
    layers.Dropout(0.5),  # 使用 Dropout 防止過擬合
    layers.Dense(10, activation='softmax')  # CIFAR-10 有 10 個類別
])

# 編譯模型
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 訓練模型
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# 評估模型
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Test accuracy: {test_acc}')

# 解凍部分卷積層，進行微調
for layer in base_model.layers[-10:]:  # 解凍最後 10 層進行微調
    layer.trainable = True

# 使用較小的學習率來微調
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 再次訓練模型
history_fine = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

# 最終評估
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'Fine-tuned test accuracy: {test_acc}')

### **微調與遷移學習的優勢**：

- **快速收斂**：通過凍結預訓練模型的大部分卷積層，我們只需要訓練少數幾層即可快速收斂，適合小數據集的情況。
- **提高精度**：微調後模型的精度通常會比不進行微調的情況下更高，這是因為模型可以更好地適應特定任務的特徵。
- **節省資源**：利用已經在大型數據集上訓練好的模型，我們不需要從頭開始訓練深度網路，節省了計算資源和時間。

---

### **結論**：

這次的實作展示了如何使用 ResNet50 進行遷移學習，並通過微調來提升模型在自定義數據集上的性能。這種方法特別適合在數據量有限或計算資源不足的情況下進行快速且高效的模型訓練。